# 垃圾邮件分类

- 如何去开展一个项目

  熟悉数据
  清洗数据
  数据分析
  选择模型
  建立模型
  
  训练模型
  模型评估
  模型优化（调参）
  
  落地上线
  
- 项目的制作流程
- 如何把手写的算法实现到真实的项目中去

In [1]:
# 朴素贝叶斯分类器训练函数
# 训练数据

def trainNB0(trainMatrix, trainCategory):
    '''
    Desc:
        分类器训练函数
    Args:
        trainMatrix:训练数据[0,1,0,1,0,2,0]
        trainCategory:训练类别 eg:[1,0,1,0]
    return:
        
    '''
    # 总文件数
    numTrainDocs = len(trainMatrix)
    # 总单词数
    numWords = len(trainMatrix[0])
    # 侮辱性文件的出现概率 = 侮辱性文件数/总文件数
    # [1，0，1，0，1，0，0]
    pAbusive = sum(trainCategory) / float(numTrainDocs)
    
    # 构造单词出现次数列表
    # p0Num 正常的统计
    # P1Num 侮辱的统计
    # [1,2,1]
    p0Num = np.ones(numWords)
    p1Num = np.ones(numWords)

    # 整个数据集单词出现的总数，根据样本/实际调查结果调整分母的值
    global p0Denom
    global p1Denom
    p0Denom = 2.0
    p1Denom = 2.0
    # 遍历总文件数

    for i in range(numTrainDocs):
        if trainCategory[i] == 1: # 垃圾邮件
            # 累加辱骂词的频次
            p1Num += trainMatrix[i]
            # 对每篇文章的辱骂的频次 进行统计汇总
            p1Denom += sum(trainMatrix[i])
        else:  # 非垃圾邮件
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    # P(单词｜类别) = p(单词1｜类别1)*p(单词2｜类别1)
    # 类别1，即侮辱性文档的[log(P(F1|C1)),log(P(F2|C1)),log(P(F3|C1)),log(P(F4|C1)),log(P(F5|C1))....]列表
    p1Vect = np.log(p1Num / p1Denom)
    # 类别0，即正常文档的[log(P(F1|C0)),log(P(F2|C0)),log(P(F3|C0)),log(P(F4|C0)),log(P(F5|C0))....]列表
    p0Vect = np.log(p0Num / p0Denom)
    return p0Vect, p1Vect, pAbusive

In [2]:
# 切分文本
def textParse(bigString):
    '''
    Desc:
        接收一个大字符串并将其解析为字符串列表
    Args:
        bigString -- 大字符串
        jieba
    Returns:
        去掉少于 2 个字符的字符串，并将所有字符串转换为小写，返回字符串列表
    '''
    import re
    # 使用正则表达式来切分句子，其中分隔符是除单词、数字外的任意字符串
    listOfTokens = re.split(r'\W+', bigString)
    return [tok.lower() for tok in listOfTokens if len(tok) > 2]

In [3]:
# 创建所有单词的集合
def createVocabList(dataSet):
    """
    获取所有单词的集合
    :param dataSet: 数据集
    :return: 所有单词的集合(即不含重复元素的单词列表)
    """
    vocabSet = set([])
    for document in dataSet:
        # 操作符 | 用于求两个集合的并集
        vocabSet = vocabSet | set(document)
    return list(vocabSet)

# 词向量转化
def setOfWords2Vec(vocabList, inputSet):
    """
    遍历查看该单词是否出现，出现该单词则将该单词置1
    :param vocabList: 所有单词集合列表
    :param inputSet: 输入数据集
    :return: 匹配列表[0,1,0,1...]，其中 1与0 表示词汇表中的单词是否出现在输入的数据集中
    """
    # 创建一个和词汇表等长的向量，并将其元素都设置为0
    returnVec = [0] * len(vocabList)  # [0,0......]
    # 遍历文档中的所有单词，如果出现了词汇表中的单词，则将输出的文档向量中的对应值设为1
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else:
            print("the word: %s is not in my Vocabulary!" % word)
    return returnVec

In [4]:
# 朴素贝叶斯分类函数
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    # 单词表*每个单词在p0/p1下出现的概率+该类别的概率（做比较）
    p1 = sum(vec2Classify * p1Vec) + np.log(pClass1)
    p0 = sum(vec2Classify * p0Vec) + np.log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else:
        return 0

In [5]:
def spamTest():
    '''
    Desc:
        对贝叶斯垃圾邮件分类器进行自动化处理。
    Args:
        none
    Returns:
        对测试集中的每封邮件进行分类，若邮件分类错误，则错误数加 1，最后返回总的错误百分比。
    '''
    # 单词表
    docList = []
    # 类别表
    classList = []
    # 文本表
    fullText = []
    # 把所有文本划分到相应的列表中
    for i in range(1, 26):
        # 切分，解析数据，并归类为 1 类别
        wordList = textParse(
            open(r'4.NaiveBayes/email/spam/%d.txt' % i,
                 encoding='ISO-8859-1').read())
        docList.append(wordList)
        classList.append(1)
        # 切分，解析数据，并归类为 0 类别
        wordList = textParse(
            open(r'4.NaiveBayes/email/ham/%d.txt' % i,
                 encoding='ISO-8859-1').read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    # 创建所有单词，词汇表
    vocabList = createVocabList(docList)
    trainingSet = list(range(50))
    # train = 50 - 10 =40
    testSet = []
    # 随机取 10 个邮件用来测试
    for i in range(10):
        # random.uniform(x, y) 随机生成一个范围为 x ~ y 的实数
        randIndex = int(random.uniform(0, len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del (trainingSet[randIndex])
    trainMat = []
    trainClasses = []
    # 训练数据
    # [0,1,0,1,0,0,1]
    # [1,0,0,1,1,1,1]
    for docIndex in trainingSet:
        trainMat.append(setOfWords2Vec(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V, p1V, pSpam = trainNB0(np.array(trainMat), np.array(trainClasses))
    errorCount = 0
    # 测试数据
    for docIndex in testSet:
        wordVector = setOfWords2Vec(vocabList, docList[docIndex])
        if classifyNB(np.array(wordVector), p0V, p1V,
                      pSpam) != classList[docIndex]:
            errorCount += 1
    print('the errorCount is: ', errorCount)
    print('the testSet length is :', len(testSet))
    print('the error rate is :', float(errorCount) / len(testSet))

In [6]:
import random
import numpy as np
import math
spamTest()


the errorCount is:  0
the testSet length is : 10
the error rate is : 0.0
